In [1]:
import os
import dask
import dask.array as da
import napari
from skimage.io import imread as skimread
import numpy as np
from glob import glob
from collections import Counter
import pycudadecon
import re
from dask.cache import Cache
cache = Cache(2e9)
cache.register()
# from cachey import Cache
# c = Cache(2e9)


In [2]:
def verbose_imread(fname):
    print(f"reading: {os.path.basename(fname)}")
    return skimread(fname)

# memreaad = c.memoize(verbose_imread)

imread = dask.delayed(verbose_imread, name="imread")

In [3]:
filenames = sorted(glob('/Volumes/extra850/mitosis/*ch0*.tif'))
fpat = re.compile('.*_ch(?P<c>\d{1}).*_stack(?P<t>\d{4})')
cs, ts = set(), set()
for f in filenames:
    gd = fpat.match(f).groupdict()
    cs.add(gd['c'])
    ts.add(gd['t'])
nc = len(cs)
nt = len(ts)

# load the first image (assume rest are same shape/dtype)
sample = imread(filenames[0]).compute()
nz, ny, nx = sample.shape
print(f"nTCZYX: {nt, nc, nz, ny, nx}")

reading: cell1_ch0_stack0000_488nm_0000000msec_0002379963msecAbs.tif
nTCZYX: (600, 1, 65, 256, 256)


/Users/talley/miniconda3/envs/all/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2133: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")


In [5]:
# Lazily evaluate imread on each path
lazy_images = [imread(path) for path in filenames]

arrays = [da.from_delayed(lazy_image,           # Construct a small Dask array
                          dtype=sample.dtype,   # for every lazy value
                          shape=sample.shape)
          for lazy_image in lazy_images]
stack = da.stack(arrays, axis=0)                # Stack all small Dask arrays into one
stack

,Array,Chunk
Bytes,5.11 GB,8.52 MB
Shape,"(600, 65, 256, 256)","(1, 65, 256, 256)"
Count,1800 Tasks,600 Chunks
Type,uint16,numpy.ndarray


In [29]:
from functools import partial
psf = '/Users/talley/Dropbox (HMS)/CBMF/lattice_sample_data/lls_PSFs/488_psf.tif'
psf = skimread(psf)

def last3dims(f):
    def func(array):
        if not array.ndim > 3:
            return array
        result = f(array[0])
        return np.expand_dims(result, 0)
    return func

def bgrd(a):
    b = a.astype('float32')
    b -= 90
    return np.clip(b, 0, 2**16).astype('uint16')

@last3dims
def deskew(a):
    print('hi')
    return pycudadecon.deskew_gpu(a, angle=31, pad_val=0)

@last3dims
def decon(a):
    return pycudadecon.decon(a, psf, background=0)

subb = stack.map_blocks(bgrd, dtype='uint16')
deskewed = subb.map_blocks(deskew, dtype='uint16')
deconvolved = deskewed.map_blocks(decon, dtype='float32')

In [30]:
with napari.gui_qt():
    napari.view_image(deconvolved)

ZMQError: Too many open files

In [11]:
deskewed[0].compute()

TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes


reading: cell1_ch0_stack0000_488nm_0000000msec_0002379963msecAbs.tif
4


IndexError: too many indices for array

In [9]:
stack

,Array,Chunk
Bytes,5.11 GB,8.52 MB
Shape,"(600, 65, 256, 256)","(1, 65, 256, 256)"
Count,1800 Tasks,600 Chunks
Type,uint16,numpy.ndarray


In [ ]:
for i in range(600):
    print(i)
    print(deconvolved[i].compute().shape)

TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes


0
(64, 256, 525)
1
reading: cell1_ch0_stack0001_488nm_0003870msec_0002383833msecAbs.tif


TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes


(64, 256, 525)
2
reading: cell1_ch0_stack0002_488nm_0007740msec_0002387703msecAbs.tif


TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes


...

(64, 256, 525)
123
reading: cell1_ch0_stack0123_488nm_0475996msec_0002855959msecAbs.tif


TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes


(64, 256, 525)
124
reading: cell1_ch0_stack0124_488nm_0479865msec_0002859828msecAbs.tif


TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes


(64, 256, 525)
125
reading: cell1_ch0_stack0125_488nm_0483735msec_0002863698msecAbs.tif
